### Peano numbers

In [1]:
abstract type Peano end

In [2]:
struct Z <: Peano end

In [3]:
struct S{T <: Peano} <: Peano end

In [4]:
typeof(Peano)

DataType

In [5]:
p0 = Z()

Z()

In [6]:
p1 = S{Z}()

S{Z}()

In [7]:
typeof(p1)

S{Z}

In [8]:
succ(x::Peano) = S{typeof(x)}()

succ (generic function with 1 method)

In [9]:
pred(x::S{X}) where {X} = X()

pred (generic function with 1 method)

In [10]:
pred(p1)

Z()

In [11]:
p2 = succ(p1)

S{S{Z}}()

In [12]:
pred(p2)

S{Z}()

In [13]:
p3 = succ(p2)

S{S{S{Z}}}()

In [14]:
succ(p0) === p1

true

### `add`

In [15]:
add(x::Z, y::Peano) = y

add (generic function with 1 method)

In [16]:
add(x::S, y::Peano) = succ(add(pred(x), y))

add (generic function with 2 methods)

In [17]:
add(p2, p3)

S{S{S{S{S{Z}}}}}()

In [18]:
@code_typed add(p1, p2)

CodeInfo(
1 ─     return $(QuoteNode(S{S{S{Z}}}()))
) => S{S{S{Z}}}

```text
ackermann(Z, n) = S(n);
ackermann(S(m), Z) = ackermann(m, S(Z));
ackermann(S(m), S(n)) = ackermann(m, a(S(m), n));
```

In [19]:
ackerman(m::Z, n::Peano) = succ(n)

ackerman (generic function with 1 method)

In [20]:
ackerman(m::S, n::Z) = ackerman(pred(m), succ(Z()))

ackerman (generic function with 2 methods)

In [21]:
ackerman(m::S, n::S) = ackerman(pred(m), ackerman(m, pred(n)))

ackerman (generic function with 3 methods)

In [22]:
ackerman(p3, p1)

S{S{S{S{S{S{S{S{S{S{S{S{S{Z}}}}}}}}}}}}}()

In [23]:
(@which ackerman(p3, p1)).specializations

svec(MethodInstance for ackerman(::S{S{S{Z}}}, ::S{Z}), MethodInstance for ackerman(::S{S{Z}}, ::S{Z}), MethodInstance for ackerman(::S{Z}, ::S{Z}), MethodInstance for ackerman(::S{Z}, ::S{S{S{Z}}}), MethodInstance for ackerman(::S{Z}, ::S{S{Z}}), MethodInstance for ackerman(::S{S{Z}}, ::S{S{S{S{S{Z}}}}}), MethodInstance for ackerman(::S{S{Z}}, ::S{S{S{S{Z}}}}), MethodInstance for ackerman(::S{S{Z}}, ::S{S{S{Z}}}), MethodInstance for ackerman(::S{S{Z}}, ::S{S{Z}}), MethodInstance for ackerman(::S{Z}, ::S{S{S{S{S{Z}}}}}), MethodInstance for ackerman(::S{Z}, ::S{S{S{S{Z}}}}), MethodInstance for ackerman(::S{Z}, ::S{S{S{S{S{S{S{Z}}}}}}}), MethodInstance for ackerman(::S{Z}, ::S{S{S{S{S{S{Z}}}}}}), MethodInstance for ackerman(::S{Z}, ::S{S{S{S{S{S{S{S{S{Z}}}}}}}}}), MethodInstance for ackerman(::S{Z}, ::S{S{S{S{S{S{S{S{Z}}}}}}}}), MethodInstance for ackerman(::S{Z}, ::S{S{S{S{S{S{S{S{S{S{S{Z}}}}}}}}}}}), MethodInstance for ackerman(::S{Z}, ::S{S{S{S{S{S{S{S{S{S{Z}}}}}}}}}}), #undef, #undef

### `repeat`

In [24]:
repeat(n::Z, a) = ()

repeat (generic function with 1 method)

In [25]:
repeat(n::S, a) = (a, repeat(pred(n), a))

repeat (generic function with 2 methods)

In [26]:
repeat(p3, 99)

(99, (99, (99, ())))

In [27]:
@code_typed repeat(p3, 99)

CodeInfo(
1 ─ %1 = Core.tuple(a, ())::Tuple{Int64, Tuple{}}
│   %2 = Core.tuple(a, %1)::Tuple{Int64, Tuple{Int64, Tuple{}}}
│   %3 = Core.tuple(a, %2)::Core.PartialStruct(Tuple{Int64, Tuple{Int64, Tuple{Int64, Tuple{}}}}, Any[Int64, Core.PartialStruct(Tuple{Int64, Tuple{Int64, Tuple{}}}, Any[Int64, Core.PartialStruct(Tuple{Int64, Tuple{}}, Any[Int64, Core.Const(())])])])
└──      return %3
) => Tuple{Int64, Tuple{Int64, Tuple{Int64, Tuple{}}}}

### `pw`

In [28]:
pw(n::Z, x) = one(x)

pw (generic function with 1 method)

In [29]:
pw(n::S, x) = x * pw(pred(n), x)

pw (generic function with 2 methods)

In [30]:
@code_typed pw(p3, 99)

CodeInfo(
1 ─ %1 = Base.mul_int(x, 1)::Int64
│   %2 = Base.mul_int(x, %1)::Int64
│   %3 = Base.mul_int(x, %2)::Int64
└──      return %3
) => Int64

In [31]:
@code_typed pw(p2, 2.0)

CodeInfo(
1 ─ %1 = Base.mul_float(x, 1.0)::Float64
│   %2 = Base.mul_float(x, %1)::Float64
└──      return %2
) => Float64

### `Val(c)`

In [32]:
Val(99)

Val{99}()

In [33]:
typeof(Val(99))

Val{99}

In [34]:
typeof(Val((1,2,(3,4))))

Val{(1, 2, (3, 4))}

### Extracting `c` from `Val(c)`

In [35]:
get_val(::Val{n}) where {n} = n

get_val (generic function with 1 method)

In [36]:
get_val(Val(99))

99

In [37]:
get_val(Val((1,2,(3,4))))

(1, 2, (3, 4))

In [38]:
get_val(Val((:A, :B, :C)))

(:A, :B, :C)

### `pw(::Val{c}, x)`

In [39]:
pw_v(::Val{0}, x) = one(x)

pw_v (generic function with 1 method)

In [40]:
pw_v(::Val{n}, x) where {n} = x * pw_v(Val(n-1), x)

pw_v (generic function with 2 methods)

In [41]:
pw_v(n::Integer, x) = pw_v(Val(n), x)

pw_v (generic function with 3 methods)

In [42]:
pw_v(3, 2.0)

8.0

In [43]:
pw_v(5, 10)

100000

In [44]:
@code_typed pw_v(Val(5), 10)

CodeInfo(
1 ─ %1 = Base.mul_int(x, 1)::Int64
│   %2 = Base.mul_int(x, %1)::Int64
│   %3 = Base.mul_int(x, %2)::Int64
│   %4 = Base.mul_int(x, %3)::Int64
│   %5 = Base.mul_int(x, %4)::Int64
└──      return %5
) => Int64

### `one_of` - static `if`

Note that `k` is decreasing, in order to induce the compiler to unfold the calls to `one_of`.

In [45]:
one_of(x, s::NTuple{N, T}) where {N, T} = one_of(x, Val(s), Val(N))

one_of (generic function with 1 method)

In [46]:
function one_of(x, ::Val{s}, ::Val{k}) where {s, k}
	if k == 0
		false
	elseif s[length(s) - k + 1] == x
		true
	else
		one_of(x, Val(s), Val(k-1))
	end
end

one_of (generic function with 2 methods)

In [47]:
one_of(:B, (:A, :B, :C))

true

In [48]:
one_of(:D, (:A, :B, :C))

false

In [49]:
@code_typed one_of(:D, (:A, :B, :C))

CodeInfo(
1 ─ %1 = invoke Main.Val(_3::Tuple{Symbol, Symbol, Symbol})::Val{_A} where _A
│   %2 = Main.one_of(x, %1, $(QuoteNode(Val{3}())))::Bool
└──      return %2
) => Bool

In [50]:
@code_native one_of(:D, Val((:A, :B, :C)), Val(3))

	.text
; ┌ @ In[46]:1 within `one_of'
	movabsq	$140255534565616, %rax          # imm = 0x7F8FC950C8F0
; │ @ In[46]:4 within `one_of'
; │┌ @ Base.jl:87 within `=='
	cmpq	%rax, %rdi
	sete	%al
	movabsq	$140255534565656, %rcx          # imm = 0x7F8FC950C918
	cmpq	%rcx, %rdi
	sete	%cl
; │└
	orb	%al, %cl
	movabsq	$140255535043832, %rax          # imm = 0x7F8FC95814F8
	cmpq	%rax, %rdi
	sete	%al
	orb	%cl, %al
; │ @ In[46]:5 within `one_of'
	retq
	nopw	%cs:(%rax,%rax)
; └


In [51]:
@code_typed one_of(:D, Val((:A, :B)), Val(2))

CodeInfo(
1 ─       goto #3 if not false
2 ─       nothing::Nothing
3 ┄ %3  = (:A === x)::Bool
└──       goto #5 if not %3
4 ─       return true
5 ─ %6  = (:B === x)::Bool
└──       goto #7 if not %6
6 ─       goto #8
7 ─       goto #8
8 ┄ %10 = φ (#6 => true, #7 => false)::Bool
└──       return %10
) => Bool

### `StaticNumbers`

In [52]:
using StaticNumbers

In [53]:
d = 2

2

In [54]:
s = static(2)

static(2)

The macro @stat makes the result of a computation a Static when all arguments are static or literals.

In [55]:
@stat s + s

static(4)

In [56]:
@stat s + 2

static(4)

In [57]:
@stat s + d

4

In [58]:
Tuple(i^2 for i in static(1:4)) # computed at compile time

(1, 4, 9, 16)

In [59]:
@code_typed Tuple(i^2 for i in static(1:4))

CodeInfo(
1 ─     invoke Base.power_by_squaring(static(1)::StaticInteger{1}, 2::Int64)::Int64
└──     return (1, 4, 9, 16)
) => NTuple{4, Int64}

In [60]:
@code_typed Tuple(i^2 for i in 1:4)

CodeInfo(
1 ─ %1 = invoke Base.collect(_2::Base.Generator{UnitRange{Int64}, var"#5#6"})::Vector{Int64}
│   %2 = Core._apply_iterate(Base.iterate, Core.tuple, %1)::Tuple{Vararg{Int64, N} where N}
└──      return %2
) => Tuple{Vararg{Int64, N} where N}

In [61]:
pw_s(n, x) = iszero(n) ? one(x) : x * pw_s(static(n - 1), x)

pw_s (generic function with 1 method)

In [62]:
pw_s(3, "a")

"aaa"

In [63]:
@code_typed pw_s(3, 10)

CodeInfo(
1 ─ %1 = (n === 0)::Bool
└──      goto #3 if not %1
2 ─      return 1
3 ─ %4 = Base.sub_int(n, 1)::Int64
│   %5 = Core.apply_type(StaticNumbers.StaticInteger, %4)::Type{StaticInteger{_A}} where _A
│   %6 = (%5)()::StaticInteger{_A} where _A
│   %7 = Main.pw_s(%6, x)::Int64
│   %8 = Base.mul_int(x, %7)::Int64
└──      return %8
) => Int64

In [64]:
@code_typed pw_s(static(3), 10)

CodeInfo(
1 ─      goto #3 if not false
2 ─      nothing::Nothing
3 ┄ %3 = Base.mul_int(x, 1)::Int64
│   %4 = Base.mul_int(x, %3)::Int64
│   %5 = Base.mul_int(x, %4)::Int64
└──      return %5
) => Int64

### Ackermann

In [65]:
function ackermann(m,n)
    if iszero(m)
        n + one(n)
    elseif iszero(n)
        ackermann(@stat(m - one(m)), one(n))
    else
        ackermann(@stat(m - one(m)), ackermann(m, n - one(n)))
    end
end

ackermann (generic function with 1 method)

In [66]:
using BenchmarkTools

In [67]:
@btime ackermann(4, 1)

  13.871 s (0 allocations: 0 bytes)


65533

In [68]:
@btime ackermann(static(4), 1)

  2.038 s (0 allocations: 0 bytes)


65533